In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

# Dataset

In [ ]:
train_dataset = load_dataset("SuryaKrishna02/aya-telugu-food-recipes", split="train")

In [ ]:
def formatting_func(example):
    output_texts = []
    for i in range(len(example)):
        text = f"# ఇన్పుట్\n{example['inputs'][i]}\n\n# లక్ష్యం\n{example['target'][i]}"
        output_texts.append(text)
    return output_texts

# Tokenizer and Model

In [ ]:
tokenizer_checkpoint = "jayasuryajsk/Llama-3-8b-Telugu-Romanized"
model_checkpoint = "jayasuryajsk/Llama-3-8b-Telugu-Romanized"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="cuda" if torch.cuda.is_available() else "cpu")

if tokenizer.pad_token is None and model.config.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Train

## Config

In [ ]:
args = SFTConfig(output_dir="tmp_trainer/aya_telugu_food_recipes")

## Run

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func
)
trainer.train()